In [36]:
# https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes, as dict.key_name, not as dict["key_name"] """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


In [37]:
import yaml

# Read config.yaml file
with open("config.yaml") as infile:
    SAVED_CFG = yaml.load(infile, Loader=yaml.FullLoader)
    SAVED_CFG = dotdict(SAVED_CFG)

# arguments setting
data_args = dotdict(SAVED_CFG.data)
model_args = dotdict(SAVED_CFG.custom_model)


In [38]:
from datasets import load_from_disk

questions = load_from_disk(data_args.test_dataset_name)
question_dataset =questions['validation']

# 240 public questions + 360 private questions = 600 total questions
question_dataset

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [39]:
print(len(question_dataset))
print(question_dataset[0])
question_dataset[0]["question"]

600
{'question': "유령'은 어느 행성에서 지구로 왔는가?", 'id': 'mrc-1-000653'}


"유령'은 어느 행성에서 지구로 왔는가?"

In [41]:
import json
import pandas as pd
# load test dataset as dataframe
with open("/opt/ml/data/wikipedia_documents.json", "r", encoding="utf-8") as reader:
    input_data = json.load(reader)
test_df = pd.DataFrame(input_data).T
print(test_df.shape)
test_df.head(2)

(60613, 8)


,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1


In [18]:
test_json = test_df.to_dict(orient="records")
test_json[0]

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [8]:
from elasticsearch import Elasticsearch
import pprint  
INDEX_NAME = "toy_index"

INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}


In [9]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [11]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-11-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-11-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

0it [00:00, ?it/s]<ipython-input-34-331b441f56ed>:5: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, id=doc_id, body=item)
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
477it [00:51,  9.50it/s]

KeyboardInterrupt: 

In [ ]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

{'_id': '1',
 '_ignored': ['text.keyword'],
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 213,
 '_source': {'author': None,
             'corpus_source': '위키피디아',
             'document_id': 1,
             'domain': None,
             'html': None,
             'text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 '
                     '다음의 조건을 만족해야 한다.\n'
                     '* (a) 영속적인 국민\n'
                     '* (b) 일정한 영토\n'
                     '* (c) 정부\n'
                     '* (d) 타국과의 관계 참여 자격.\n'
                     '특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 '
                     '중요한 역할을 할 수 있다.  이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 '
                     '자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 '
                     '되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 '
                     '견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n'
                     '\n'
                     '위 기준에 논거하여 이 목록

In [ ]:
tv = es.termvectors(index=INDEX_NAME, id=2,body={"fields" : ["content","title"]})

In [ ]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'toy_index',
 '_type': '_doc',
 '_version': 2,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 312,
                                                 'sum_doc_freq': 794,
                                                 'sum_ttf': 794},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 14}


In [ ]:
query = test_dataset[0]["question"]
res = es.search(index=INDEX_NAME, q=query, size=10)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '100',
                    '_ignored': ['text.keyword'],
                    '_index': 'toy_index',
                    '_score': 4.098283,
                    '_source': {'author': None,
                                'corpus_source': '위키피디아',
                                'document_id': 100,
                                'domain': None,
                                'html': None,
                                'text': '블랙데이가 언제부터 어떻게 시작됐는지는 정확히 알 수 없다. '
                                        '처음으로 언급된 것은 1990년대 신문 기사들로, 어느 신문은 '
                                        '블랙데이에 대해 “애인이 없는 남녀 청소년들이 검은색 옷을 입고 '
                                        '끼리끼리 몰려다니다 짜장면과 블랙 커피, 아메리카노등 검은색 계통의 '
                                        '음식을 먹으며 서로 위로하는 날”로 해석하기도 했다\n'
                                        '이 때문에 한쪽에서는 청소년 사이에서 시작된 유행이 2000년대 '
                                        '들어 

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))


Doc ID: '100'  Score:  4.10
Doc ID: '84'  Score:  3.94
Doc ID: '16'  Score:  3.72
Doc ID: '67'  Score:  3.46
Doc ID: '174'  Score:  2.78
